In [ ]:
!wget https://github.com/MorvanZhou/PyTorch-Tutorial/raw/master/tutorial-contents-notebooks/mnist/processed/training.pt
!wget https://github.com/MorvanZhou/PyTorch-Tutorial/raw/master/tutorial-contents-notebooks/mnist/processed/test.pt

--2021-09-28 03:53:43--  https://github.com/MorvanZhou/PyTorch-Tutorial/raw/master/tutorial-contents-notebooks/mnist/processed/training.pt
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MorvanZhou/PyTorch-Tutorial/master/tutorial-contents-notebooks/mnist/processed/training.pt [following]
--2021-09-28 03:53:43--  https://raw.githubusercontent.com/MorvanZhou/PyTorch-Tutorial/master/tutorial-contents-notebooks/mnist/processed/training.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47520431 (45M) [application/octet-stream]
Saving to: ‘training.pt’

training.pt         100%[===================>]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam

In [ ]:
device = torch.device('cuda')

In [ ]:
X_train, y_train = torch.load('training.pt')
X_test, y_test = torch.load('test.pt')

In [ ]:
X_train, X_test = X_train.reshape(-1,1,28,28), X_test.reshape(-1,1,28,28)

In [ ]:
train_td = TensorDataset(X_train, y_train)
test_td = TensorDataset(X_test, y_test)

In [ ]:
train_loader = DataLoader(train_td, batch_size = 512, shuffle=True)
test_loader = DataLoader(test_td, batch_size = 512, shuffle=True)

In [ ]:
y_train

tensor([5, 0, 4,  ..., 5, 6, 8])

In [ ]:
class MNIST_CNN(nn.Module):

    def __init__(self):
        
        super(MNIST_CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size = 3, padding = 1)
        self.bn1 = nn.BatchNorm2d(num_features = 16)
        self.drop = nn.Dropout2d(p = 0.4)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, padding = 1)
        self.bn2 = nn.BatchNorm2d(num_features = 32)
        self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1)
        self.bn3 = nn.BatchNorm2d(num_features = 64)
        self.conv4 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=128, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        #print("After conv1 and pool 1",x.size())

        x = self.bn1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        #print("After conv2 and pool 2",x.size())
        
        x = self.bn2(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        #print("After conv3 and pool 3",x.size())
        
        x = self.bn3(x)
        x = self.conv4(x)
        x = F.relu(x)
        x = self.pool(x)
        #print("After conv4 and pool 4",x.size())

        x = x.reshape(x.size()[0],-1)
        #print(x.size())
        x = self.fc1(x)
        s = F.relu(x)
        x = self.drop(x)
        x = self.fc2(x)

        return x

In [ ]:
net = MNIST_CNN()

In [ ]:
net(X_train[:2].float())

After conv4 and pool 4 torch.Size([2, 128, 1, 1])
torch.Size([2, 128])


tensor([[-0.1551,  0.1759,  0.0065, -0.1747,  0.2149,  0.3240,  0.2247,  0.2662,
          0.1571, -0.4934],
        [ 0.0932,  0.2717,  0.3821,  0.1651,  0.1820,  0.1163,  0.1672,  0.2105,
          0.4310, -0.3020]], grad_fn=<AddmmBackward>)

In [ ]:
train_loader

In [ ]:
net = net.to(device)
opt = Adam(net.parameters())
n_epochs=5

for epoch in range(n_epochs):
    training_loss=0.0

    for batch, target in train_loader:
        batch = batch.float().to(device)
        target = target.long().to(device)

        opt.zero_grad()

        output = net(batch)
        loss = F.cross_entropy(output, target)
        loss.backward()

        training_loss += loss.item()

        opt.step()
    val_output = net(X_test.float().to(device))
    val_loss = F.cross_entropy(val_output, y_test.long().to(device))
    print(f"Training loss: {training_loss} Valdiation loss: {val_loss.item()}")

After conv4 and pool 4 torch.Size([512, 128, 1, 1])
torch.Size([512, 128])
After conv4 and pool 4 torch.Size([512, 128, 1, 1])
torch.Size([512, 128])
After conv4 and pool 4 torch.Size([512, 128, 1, 1])
torch.Size([512, 128])
After conv4 and pool 4 torch.Size([512, 128, 1, 1])
torch.Size([512, 128])
After conv4 and pool 4 torch.Size([512, 128, 1, 1])
torch.Size([512, 128])
After conv4 and pool 4 torch.Size([512, 128, 1, 1])
torch.Size([512, 128])
After conv4 and pool 4 torch.Size([512, 128, 1, 1])
torch.Size([512, 128])
After conv4 and pool 4 torch.Size([512, 128, 1, 1])
torch.Size([512, 128])
After conv4 and pool 4 torch.Size([512, 128, 1, 1])
torch.Size([512, 128])
After conv4 and pool 4 torch.Size([512, 128, 1, 1])
torch.Size([512, 128])
After conv4 and pool 4 torch.Size([512, 128, 1, 1])
torch.Size([512, 128])
After conv4 and pool 4 torch.Size([512, 128, 1, 1])
torch.Size([512, 128])
After conv4 and pool 4 torch.Size([512, 128, 1, 1])
torch.Size([512, 128])
After conv4 and pool 4 to